# COSMOS2020 public redshifts

We want to use the public COSMOS2020 redhshifts. These are downloaded from:

https://cosmos2020.calet.org/catalogues/PZ/

Here we simply retrieve the medians from the p of z array and add in help_id from the masterlist cross id table

In [1]:
from astropy.table import Table,join
from astropy.io import fits
import numpy as np

In [2]:
redshift_arrray=fits.open('./data/COSMOS2020_CLASSIC_R1_v2.0_EAZY_CZ.fits')[1].data
redshift_arrray.shape

(1720701, 52)

In [3]:
help_cat=Table.read('../../dmu1/dmu1_ml_COSMOS/data/dr2_master_list_cross_ident_cosmos_20210120.fits')['cosmos_id','help_id']
help_cat=help_cat[help_cat['cosmos_id']>0]

In [4]:
help_cat[:5]

cosmos_id,help_id
int64,bytes27
471237,HELP_J100003.524+015224.355
1222628,HELP_J100059.943+023449.651
1458140,HELP_J095720.800+024839.961
234355,HELP_J095822.549+013819.471
568813,HELP_J095756.235+015804.767


In [5]:
# code from https://cosmos2020.calet.org/catalogues/PZ/eazy_zcdf_pdf.txt
SIGMAS = np.array([-5. , -4.8, -4.6, -4.4, -4.2, -4. , -3.8, -3.6, -3.4, -3.2, -3. ,
       -2.8, -2.6, -2.4, -2.2, -2. , -1.8, -1.6, -1.4, -1.2, -1. , -0.8,
       -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,
        1.6,  1.8,  2. ,  2.2,  2.4,  2.6,  2.8,  3. ,  3.2,  3.4,  3.6,
        3.8,  4. ,  4.2,  4.4,  4.6,  4.8,  5. ])

def zcdf_to_pdf(zcdf, sigmas=SIGMAS):
    """
    Convert compressed redshift CDF to PDF
    
    Parameters
    ----------
    zcdf : array-like
        Redshifts where ``cdf(z)`` for ``normal_cdf(mu=0, sigmas)``
    
    sigmas : array-like
        Values where ``normal_cdf`` is evaluated
    Returns
    -------
    pdf : array-like
        Converted ``p(z)``
        
    """
    import scipy.stats
    # Evaluate normal distribution at these values to get PDF(x)
    pdf_y = scipy.stats.norm.pdf(sigmas)
    return pdf_y / np.trapz(pdf_y, zcdf)

In [6]:
#first row is sigma values 26th is 0 with -99 for id column
redshift_arrray[0]

array([-99. ,  -5. ,  -4.8,  -4.6,  -4.4,  -4.2,  -4. ,  -3.8,  -3.6,
        -3.4,  -3.2,  -3. ,  -2.8,  -2.6,  -2.4,  -2.2,  -2. ,  -1.8,
        -1.6,  -1.4,  -1.2,  -1. ,  -0.8,  -0.6,  -0.4,  -0.2,   0. ,
         0.2,   0.4,   0.6,   0.8,   1. ,   1.2,   1.4,   1.6,   1.8,
         2. ,   2.2,   2.4,   2.6,   2.8,   3. ,   3.2,   3.4,   3.6,
         3.8,   4. ,   4.2,   4.4,   4.6,   4.8,   5. ])

In [7]:
len(SIGMAS)

51

In [8]:
SIGMAS[25]

0.0

In [9]:
redshift_arrray[1:,26].shape

(1720700,)

In [10]:
redshift_medians=redshift_arrray[1:,26]

In [11]:
redshift_medians

array([ 0.140395  , 11.8382597 ,  0.        , ...,  6.59002638,
        5.09726191,  2.88171339])

In [12]:
redshift_arrray[-1,0]

1720700.0

In [13]:
redshift=Table()
redshift['cosmos_id']=redshift_arrray[1:,0] 
redshift['cosmos_id']=redshift['cosmos_id'].astype(int)
redshift['redshift']=redshift_arrray[1:,26] 
redshift=redshift[redshift['cosmos_id']>0] # One string 'NaN' gets switched to a negative integer

In [14]:
redshift=join(redshift,help_cat,join_type='left')

In [15]:
redshift[:5]

cosmos_id,redshift,help_id
int64,float64,bytes27
1,0.14039500057697296,HELP_J100402.048+012353.796
2,11.83825969696045,HELP_J100406.918+012415.651
3,0.0,HELP_J100401.849+012354.379
4,1.8111978769302368,HELP_J100038.969+012329.823
5,1.3386461734771729,HELP_J095809.643+012329.529


In [16]:
len(redshift)

1720699

In [17]:
redshift.write('./data/COSMOS2020_eazy_classic_with_help_id.fits',overwrite=True)